In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random as rd
from datetime import datetime, timedelta

import math
import gurobipy as gp
from gurobipy import GRB

[df 정의]
df1 : 아직 matching이 안된 호출이 담긴 dataframe
df2 : 1순위 타깃 동승객의 matching을 진행해주는 dataframe (step1 dataframe)
df3 : matching이 완료된 호출이 담긴 dataframe

# MATCHING ALGORITHM

### step0) 동승 호출 randomly 생성

In [99]:
now = datetime.now()
num_user = 50

# 출발 희망 시각[변환 전]
start = [ rd.randrange(60*17, 60*19, 5) for i in range(num_user)]
# 출발 희망 시각[변환 후]
start_time = [ datetime(now.year, now.month, now.day, hour=timedelta(minutes=a).seconds // 3600, minute=(timedelta(minutes=a).seconds // 60) % 60).strftime('%Y-%m-%d %H:%M:%S') for a in start]

# 대기 가능 시간
wait = [ rd.randrange(10, 60,10) for i in range(num_user)]

# 최대 대기 시각[변환 전]
tolerance = [start[i] + wait[i] for i in range(num_user)]
# 최대 대기기 시각[변환 후]
tolerance_time = [ datetime(now.year, now.month, now.day, hour=timedelta(minutes=a).seconds // 3600, minute=(timedelta(minutes=a).seconds // 60) % 60).strftime('%Y-%m-%d %H:%M:%S') for a in tolerance]

# 우선순위
priority = [ i for i in range(1,num_user+1)]

# range_time
range_time = [(start[i], tolerance[i]) for i in range(num_user)]

# ID
ID = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','AA','BB','CC','DD','EE','FF','GG','HH','II','JJ','KK','LL','MM','NN','OO','PP','QQ','RR','SS','TT','UU','VV','WW','XX','YY','ZZ']
id = ID[:num_user]

# 출발지 lat(위도)&lon(경도) [0,1000 사이의 난수] '도'가 아니라 가시를 위해 '초'로 표시하였다.
# start_lat = [round(1000*rd.random(),4) for i in range(num_user)]
# start_lon = [round(1000*rd.random(),4) for i in range(num_user)]
start_lat = [rd.randrange(100, 400, 100) for i in range(num_user)]
start_lon = [rd.randrange(100, 400, 100) for i in range(num_user)]
start_loc = [(start_lat[i], start_lon[i]) for i in range(num_user)]

# 출발지 lat(위도)&lon(경도) [0,1000 사이의 난수]
des_lat = [round(1000*rd.random(),4) for i in range(num_user)]
des_lon = [round(1000*rd.random(),4) for i in range(num_user)]
des_loc = [(des_lat[i], des_lon[i]) for i in range(num_user)]

In [217]:
df1 = pd.DataFrame(dict(zip(['ID','출발희망시각','최대대기시각','우선순위','시간범위','출발위치','도착위치'],[id, start_time, tolerance_time, priority, range_time, start_loc, des_loc])))
df1 = df1.sort_values(by=['출발희망시각','최대대기시각']) # sorting
df1 = df1.set_index('우선순위')
df1

,ID,출발희망시각,최대대기시각,시간범위,출발위치,도착위치
우선순위,,,,,,
15,O,2023-03-10 17:00:00,2023-03-10 17:20:00,"(1020, 1040)","(300, 200)","(8.7582, 98.3835)"
46,TT,2023-03-10 17:00:00,2023-03-10 17:20:00,"(1020, 1040)","(300, 300)","(693.7091, 31.8403)"
44,RR,2023-03-10 17:00:00,2023-03-10 17:30:00,"(1020, 1050)","(100, 300)","(91.0434, 346.785)"
24,X,2023-03-10 17:00:00,2023-03-10 17:50:00,"(1020, 1070)","(200, 200)","(186.5404, 328.1395)"
27,AA,2023-03-10 17:00:00,2023-03-10 17:50:00,"(1020, 1070)","(200, 200)","(583.882, 177.3371)"
41,OO,2023-03-10 17:05:00,2023-03-10 17:55:00,"(1025, 1075)","(200, 100)","(666.1148, 602.7884)"
49,WW,2023-03-10 17:15:00,2023-03-10 17:45:00,"(1035, 1065)","(300, 100)","(433.9611, 842.1573)"
35,II,2023-03-10 17:20:00,2023-03-10 17:50:00,"(1040, 1070)","(300, 300)","(352.1314, 978.629)"
34,HH,2023-03-10 17:20:00,2023-03-10 18:00:00,"(1040, 1080)","(300, 300)","(298.6848, 738.5939)"


### step1) 타깃 동승객(1순위)의 시간범위와 겹치지 않는 호출 탈락

In [218]:
df2 = df1.copy()
target_index = 1
target_info = df2.loc[target_index]

start, end = df2.loc[target_index]['시간범위']
for index, row in df2.iterrows():
  st, en = df2.loc[index]['시간범위']
  if start>=en or end<=st:
    df2 = df2.drop(index, axis=0)
df2

,ID,출발희망시각,최대대기시각,시간범위,출발위치,도착위치
우선순위,,,,,,
24,X,2023-03-10 17:00:00,2023-03-10 17:50:00,"(1020, 1070)","(200, 200)","(186.5404, 328.1395)"
27,AA,2023-03-10 17:00:00,2023-03-10 17:50:00,"(1020, 1070)","(200, 200)","(583.882, 177.3371)"
41,OO,2023-03-10 17:05:00,2023-03-10 17:55:00,"(1025, 1075)","(200, 100)","(666.1148, 602.7884)"
49,WW,2023-03-10 17:15:00,2023-03-10 17:45:00,"(1035, 1065)","(300, 100)","(433.9611, 842.1573)"
35,II,2023-03-10 17:20:00,2023-03-10 17:50:00,"(1040, 1070)","(300, 300)","(352.1314, 978.629)"
34,HH,2023-03-10 17:20:00,2023-03-10 18:00:00,"(1040, 1080)","(300, 300)","(298.6848, 738.5939)"
16,P,2023-03-10 17:25:00,2023-03-10 18:05:00,"(1045, 1085)","(300, 200)","(307.1165, 815.1679)"
19,S,2023-03-10 17:30:00,2023-03-10 18:00:00,"(1050, 1080)","(200, 200)","(153.3404, 186.8074)"
1,A,2023-03-10 17:30:00,2023-03-10 18:10:00,"(1050, 1090)","(300, 100)","(97.3956, 572.4011)"


### step2) 타킷 동승객의 출발지&도착지 기준 반경 $m$km 이내에 벡터가 위치한 호출만 남긴다.

In [219]:
# A의 위치정보
start_lat, start_lon = df2.loc[target_index]['출발위치']
end_lat, end_lon = df2.loc[target_index]['도착위치']

m = 20 # 20초
m = m*1000/37 # km를 초로 치환, 1초는 37m이다

# 직선의 방정식
def line_eq(x1, y1, x2, y2):
  m = (y2 - y1) / (x2 - x1)
  b = y1 - m * x1
  A, B, C = -m, 1, -b
  return A, B, C

# 직선과 A 사이의 거리
def distance(x1, y1, a, b, c):
  dist = abs(a*x1 + b*y1 + c) / math.sqrt(a*a + b*b)
  return dist

for index, row in df2.iterrows():
  st_lat, st_lon = df2.loc[index]['출발위치']
  en_lat, en_lon = df2.loc[index]['도착위치']

  # 각 ID의 벡터
  a, b, c = line_eq(st_lat, st_lon, en_lat, en_lon)
  # 각 ID의 벡터와 A 출발점 사이의 거리
  dist_from_st = distance(start_lat, start_lon, a, b, c)
  # 각 ID의 벡터와 B 출발점 사이의 거리
  dist_from_en = distance(end_lat, end_lon, a, b, c)

  # print(dist_from_st, dist_from_en)

  if dist_from_st >= m and dist_from_en >= m:
    df2 = df2.drop(index, axis=0)

df2

,ID,출발희망시각,최대대기시각,시간범위,출발위치,도착위치
우선순위,,,,,,
24,X,2023-03-10 17:00:00,2023-03-10 17:50:00,"(1020, 1070)","(200, 200)","(186.5404, 328.1395)"
27,AA,2023-03-10 17:00:00,2023-03-10 17:50:00,"(1020, 1070)","(200, 200)","(583.882, 177.3371)"
41,OO,2023-03-10 17:05:00,2023-03-10 17:55:00,"(1025, 1075)","(200, 100)","(666.1148, 602.7884)"
49,WW,2023-03-10 17:15:00,2023-03-10 17:45:00,"(1035, 1065)","(300, 100)","(433.9611, 842.1573)"
35,II,2023-03-10 17:20:00,2023-03-10 17:50:00,"(1040, 1070)","(300, 300)","(352.1314, 978.629)"
34,HH,2023-03-10 17:20:00,2023-03-10 18:00:00,"(1040, 1080)","(300, 300)","(298.6848, 738.5939)"
16,P,2023-03-10 17:25:00,2023-03-10 18:05:00,"(1045, 1085)","(300, 200)","(307.1165, 815.1679)"
19,S,2023-03-10 17:30:00,2023-03-10 18:00:00,"(1050, 1080)","(200, 200)","(153.3404, 186.8074)"
1,A,2023-03-10 17:30:00,2023-03-10 18:10:00,"(1050, 1090)","(300, 100)","(97.3956, 572.4011)"


### Step3) 벡터 빼기 연산을 통해 타깃 동승객과 방향이 같은 만족하는 호출만 남긴다.

In [220]:
# # A의 위치정보 => 벡터 빼기
# start_lat, start_lon = df1.loc[1]['출발위치']
# end_lat, end_lon = df1.loc[1]['도착위치']
#
# a_vec = [start_lat-end_lat, start_lon-end_lon]
# print(f'타깃 동승자의 vector: {a_vec}')
# for index, row in df1.iterrows():
#   st_lat, st_lon = df1.loc[index]['출발위치']
#   en_lat, en_lon = df1.loc[index]['도착위치']
#   i_vec = [st_lat-en_lat, st_lon-en_lon]
#
#   if max(math.sqrt(a_vec[0]**2 + a_vec[1]**2),math.sqrt(i_vec[0]**2 + i_vec[1]**2)) < math.sqrt((a_vec[0]-i_vec[0])**2+(a_vec[1]-i_vec[1])**2):
#     df1 = df1.drop(index, axis=0)
#
# df2 = df1.copy()
# df1

In [221]:
# A의 위치정보 => 코사인 유사도
start_lat, start_lon = df2.loc[target_index]['출발위치']
end_lat, end_lon = df2.loc[target_index]['도착위치']
a_vec = np.array([start_lat-end_lat, start_lon-end_lon])
print(f'타깃 동승자의 vector: {a_vec}')

def get_angle(v1, v2):
    """두 벡터 사이각을 계산하는 함수"""
    cos_sim = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

    if np.linalg.norm(v1) == 0 or np.linalg.norm(v2) == 0:
        return np.nan  # 벡터의 크기가 0이면 NaN 반환

    if np.abs(cos_sim) > 1:
        return np.nan  # 코사인 유사도가 1보다 크거나 -1보다 작으면 NaN 반환

    angle = np.arccos(cos_sim)
    return np.degrees(angle)  # 라디안을 도로 변환하여 반환

for index, row in df2.iterrows():
  st_lat, st_lon = df2.loc[index]['출발위치']
  en_lat, en_lon = df2.loc[index]['도착위치']
  i_vec = np.array([st_lat-en_lat, st_lon-en_lon])

  if get_angle(a_vec, i_vec) > 90:
    df2 = df2.drop(index, axis=0)
  else:
    print(get_angle(a_vec, i_vec))

타깃 동승자의 vector: [ 202.6044 -472.4011]
17.217346134403645
66.04598259077657
33.445477147419275
27.606389701195756
23.04181853452965
23.876419521621152
82.57416403089651
1.2074182697257333e-06
31.44182806575945
89.63545997334536
27.27817467362322
75.64370500962653
12.819396400929664
44.36879011270245
82.8360272367358
1.9982558322530994
36.37177793950096
39.5426684966803
2.8551842245921715
35.221019307080184
47.736029408388525
13.092214623472508
79.43775887928511
52.588654035871336
22.480971518592238
42.00645853770518


### Step4) 타깃 동승자와 각각의 후보들을 경유했을 때의 실제경로의 운행시간과 비용을 바탕으로 최적의 경유지를 탐색한다.

In [222]:
start_lat, start_lon = df2.loc[target_index]['출발위치']
end_lat, end_lon = df2.loc[target_index]['도착위치']

# 자기자신과 매칭 방지를 위해 제거
df2 = df2.drop(target_index, axis=0)

In [224]:
epoch = 4
'''
우선 x좌표를 기준으로 sorting하여 경유지를 설정하였다. 추후 경유지API 가져와 변경 예정
[2021년 서울 차량 통행속도 분석]에 따르면 지난해 도심 통행속도는 시속 23㎞'라는 정보를 활용해 시간을 산출하였다.
1초 사이의 거리는 0.037 km = 37 m 이므로 이를 기반으로 거리를 산출하였다.
'''

def distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    dist = math.sqrt(dx*dx + dy*dy)
    return dist

for index, row in df2.iterrows():
  st_lat, st_lon = start_lat, start_lon
  en_lat, en_lon = df2.loc[index]['도착위치']
  # lat_lon = {
  #     start_lat: start_lon,
  #     end_lat: end_lon,
  #     st_lat: st_lon,
  #     en_lat: en_lon
  #     }
  lat_lon = {
    'a_start': df2.loc[index]['출발위치'],
    'a_end': df2.loc[index]['도착위치'],
    'i_start': (start_lat, start_lon),
    'i_end': (end_lat, end_lon)
      }
  # 원점에서 떨어진 거리순으로 정렬
  dis_lat_lon = {
      distance(0, 0, start_lat, start_lon): 'a_start', # a 시작지점
      distance(0, 0, end_lat, end_lon): 'a_end', # a 끝지점
      distance(0, 0, st_lat, st_lon): 'i_start', # i 시작지점
      distance(0, 0, en_lat, en_lon): 'i_end' # i 끝지점
      }


  dis = sorted([distance(0, 0, start_lat, start_lon), distance(0, 0, end_lat,end_lon), distance(0, 0, st_lat, st_lon), distance(0, 0, en_lat, en_lon)])
  # print(dis)
  routing_lat_lon = [ dis_lat_lon[key] for key in dis]
  print(routing_lat_lon) # 승하차순서
  routing_location = [ lat_lon[key] for key in routing_lat_lon]
  print(routing_location) # 이동경로

  routing_lat = [lat for lat,lon in [ lat_lon[key] for key in routing_lat_lon]]
  routing_lon = [lon for lat,lon in [ lat_lon[key] for key in routing_lat_lon]]

  print('routing_lat: ',routing_lat)
  print('routing_lon: ',routing_lon)
  print(f'A start: {start_lat, start_lon} A end: {end_lat, end_lon}')
  print(f'I start: {st_lat, st_lon} I end: {en_lat, en_lon}')

  # 경유 node
  if routing_lat.index(start_lat) > routing_lat.index(end_lat) or routing_lat.index(st_lat) > routing_lat.index(en_lat):
    routing_lat = routing_lat[::-1]
    routing_lon = routing_lon[::-1]
    a_routing_idx = [routing_lat.index(start_lat), routing_lat.index(end_lat)]
    i_routing_idx = [routing_lat.index(st_lat), routing_lat.index(en_lat)]


  else:
    a_routing_idx = [routing_lat.index(start_lat), routing_lat.index(end_lat)]
    i_routing_idx = [routing_lat.index(st_lat), routing_lat.index(en_lat)]


  a_route_dist = 0
  a_dist = 0
  start_point = a_routing_idx[0]
  end_point = a_routing_idx[1]
  st_point = i_routing_idx[0]
  en_point = i_routing_idx[1]

  print(start_point<end_point)


  # 0>1 , 2>3 인 경우 제외
  if (start_point, end_point) == (0,1) or (start_point, end_point) == (2,3):
    df2 = df2.drop(index, axis=0)
    print('wrong match : tp1')
    print()
  elif start_point > end_point or st_point > en_point:
    df2 = df2.drop(index, axis=0)
    print('wrong match : tp2')
    print()

  else:
    print(start_point, end_point)

    # 타킷경유시간(분)
    point = start_point
    try:
      for i in range(epoch):
        # a가 노드
        x1 = routing_lat[point]
        y1 = routing_lon[point]
        # 다음 노드
        point += 1

        x2 = routing_lat[point]
        y2 = routing_lon[point]
        dist_btn_node = distance(x1, y1, x2, y2) * 0.037 # 초(각도) >> km(거리)
        a_route_dist += dist_btn_node
        if point==end_point:
          break
    except:
      pass

    a_route_time = 60/23 * a_route_dist # km(거리) >> 분(시간)
    df2.loc[index,'타깃경유시간(분)'] = a_route_time
    df2.loc[index,'타깃경유거리(km)'] = a_route_dist

    # 타킷원래시간(분)
    # a 출발
    x1 = routing_lat[start_point]
    y1 = routing_lon[start_point]
    # a 도착
    x2 = routing_lat[end_point]
    y2 = routing_lon[end_point]
    a_dist = distance(x1, y1, x2, y2) * 0.037
    a_time = 60/23 * a_dist
    df2.loc[index,'타깃원래시간(분)'] = a_time
    df2.loc[index,'타깃원래거리(km)'] = a_dist


    i_route_dist = 0
    i_dist = 0
    start_point = i_routing_idx[0]
    end_point = i_routing_idx[1]
    print(start_point, end_point)

    # 경유시간(분)
    point = start_point
    try:
      for i in range(epoch):
        # a가 노드
        x1 = routing_lat[point]
        y1 = routing_lon[point]
        # 다음 노드
        point += 1
        x2 = routing_lat[point]
        y2 = routing_lon[point]
        dist_btn_node = distance(x1, y1, x2, y2) * 0.037 # 초(각도) >> km(거리)
        i_route_dist += dist_btn_node
        if point==end_point:
          break
    except:
      pass

    i_route_time = 60/23 * i_route_dist # km(거리) >> 분(시간)
    df2.loc[index,'경유시간(분)'] = i_route_time
    df2.loc[index,'경유거리(km)'] = i_route_dist

    # 원래시간(분)
    # i 출발
    x1 = routing_lat[start_point]
    y1 = routing_lon[start_point]
    # i 도착
    x2 = routing_lat[end_point]
    y2 = routing_lon[end_point]
    i_dist = distance(x1, y1, x2, y2) * 0.037
    i_time = 60/23 * i_dist
    df2.loc[index,'원래시간(분)'] = i_time
    df2.loc[index,'원래거리(km)'] = i_dist

    print(routing_lat, routing_lon)
    print()


    total_route_dist = 0
    point = 0

    # 전체체경유시간(분)
    try:
      for i in range(epoch):
        # a가 노드
        x1 = routing_lat[point]
        y1 = routing_lon[point]
        # 다음 노드
        point += 1

        x2 = routing_lat[point]
        y2 = routing_lon[point]
        dist_btn_node = distance(x1, y1, x2, y2) * 0.037 # 초(각도) >> km(거리)
        total_route_dist += dist_btn_node
        if point==4:
          break
    except:
      pass

    total_time = 60/23 * total_route_dist
    df2.loc[index,'총이동시간(분)'] = total_time
    df2.loc[index,'총경유거리(km)'] = total_route_dist

    df2.loc[index,'이동경로'] = ''
    df2.loc[index,'승하차순서'] = ''
    df2.at[index,'이동경로'] = routing_location
    df2.at[index,'승하차순서'] = routing_lat_lon

df2

['i_start', 'i_start', 'i_end', 'a_end']
[(300, 100), (300, 100), (97.3956, 572.4011), (186.5404, 328.1395)]
routing_lat:  [300, 300, 97.3956, 186.5404]
routing_lon:  [100, 100, 572.4011, 328.1395]
A start: (300, 100) A end: (97.3956, 572.4011)
I start: (300, 100) I end: (186.5404, 328.1395)
True
0 2
0 3
[300, 300, 97.3956, 186.5404] [100, 100, 572.4011, 328.1395]

['i_start', 'i_start', 'a_end', 'i_end']
[(300, 100), (300, 100), (666.1148, 602.7884), (97.3956, 572.4011)]
routing_lat:  [300, 300, 666.1148, 97.3956]
routing_lon:  [100, 100, 602.7884, 572.4011]
A start: (300, 100) A end: (97.3956, 572.4011)
I start: (300, 100) I end: (666.1148, 602.7884)
True
0 3
0 2
[300, 300, 666.1148, 97.3956] [100, 100, 602.7884, 572.4011]

['i_start', 'i_start', 'a_end', 'i_end']
[(300, 100), (300, 100), (433.9611, 842.1573), (97.3956, 572.4011)]
routing_lat:  [300, 300, 433.9611, 97.3956]
routing_lon:  [100, 100, 842.1573, 572.4011]
A start: (300, 100) A end: (97.3956, 572.4011)
I start: (300, 100)

,ID,출발희망시각,최대대기시각,시간범위,출발위치,도착위치,타깃경유시간(분),타깃경유거리(km),타깃원래시간(분),타깃원래거리(km),경유시간(분),경유거리(km),원래시간(분),원래거리(km),총이동시간(분),총경유거리(km),이동경로,승하차순서
우선순위,,,,,,,,,,,,,,,,,,
24,X,2023-03-10 17:00:00,2023-03-10 17:50:00,"(1020, 1070)","(200, 200)","(186.5404, 328.1395)",49.613614,19.018552,49.613614,19.018552,74.711220,28.639301,24.593298,9.427431,74.711220,28.639301,"[(300, 100), (300, 100), (97.3956, 572.4011), ...","[i_start, i_start, i_end, a_end]"
41,OO,2023-03-10 17:05:00,2023-03-10 17:55:00,"(1025, 1075)","(200, 100)","(666.1148, 602.7884)",115.004883,44.085205,49.613614,19.018552,60.032815,23.012579,60.032815,23.012579,115.004883,44.085205,"[(300, 100), (300, 100), (666.1148, 602.7884),...","[i_start, i_start, a_end, i_end]"
49,WW,2023-03-10 17:15:00,2023-03-10 17:45:00,"(1035, 1065)","(300, 100)","(433.9611, 842.1573)",114.424551,43.862744,49.613614,19.018552,72.791922,27.903570,72.791922,27.903570,114.424551,43.862744,"[(300, 100), (300, 100), (433.9611, 842.1573),...","[i_start, i_start, a_end, i_end]"
35,II,2023-03-10 17:20:00,2023-03-10 17:50:00,"(1040, 1070)","(300, 300)","(352.1314, 978.629)",131.237230,50.307605,49.613614,19.018552,84.955943,32.566445,84.955943,32.566445,131.237230,50.307605,"[(300, 100), (300, 100), (352.1314, 978.629), ...","[i_start, i_start, a_end, i_end]"
34,HH,2023-03-10 17:20:00,2023-03-10 18:00:00,"(1040, 1080)","(300, 300)","(298.6848, 738.5939)",86.833529,33.286186,49.613614,19.018552,61.638325,23.628024,61.638325,23.628024,86.833529,33.286186,"[(300, 100), (300, 100), (298.6848, 738.5939),...","[i_start, i_start, a_end, i_end]"
16,P,2023-03-10 17:25:00,2023-03-10 18:05:00,"(1045, 1085)","(300, 200)","(307.1165, 815.1679)",99.997734,38.332465,49.613614,19.018552,69.032667,26.462522,69.032667,26.462522,99.997734,38.332465,"[(300, 100), (300, 100), (307.1165, 815.1679),...","[i_start, i_start, a_end, i_end]"
22,V,2023-03-10 17:30:00,2023-03-10 18:20:00,"(1050, 1100)","(200, 200)","(302.677, 910.0534)",116.329489,44.592971,49.613614,19.018552,78.188190,29.972139,78.188190,29.972139,116.329489,44.592971,"[(300, 100), (300, 100), (302.677, 910.0534), ...","[i_start, i_start, a_end, i_end]"
31,EE,2023-03-10 17:40:00,2023-03-10 18:10:00,"(1060, 1090)","(100, 100)","(506.4641, 277.3954)",49.613614,19.018552,49.613614,19.018552,98.294017,37.679373,26.273876,10.071652,98.294017,37.679373,"[(300, 100), (300, 100), (97.3956, 572.4011), ...","[i_start, i_start, i_end, a_end]"
20,T,2023-03-10 17:40:00,2023-03-10 18:30:00,"(1060, 1110)","(100, 100)","(155.6581, 883.2667)",107.402995,41.171148,49.613614,19.018552,76.875267,29.468852,76.875267,29.468852,107.402995,41.171148,"[(300, 100), (300, 100), (155.6581, 883.2667),...","[i_start, i_start, a_end, i_end]"


### Step5) 지연시간 비율이 가장 적은 최종 동승객(1명) select

In [225]:
ratio_index = {}
len_df2 = len(df2)

alpha_min = 0.10 # 동승객 최소 할인율
beta_min = 0.10 # 동승객 최소 추가 할인율
gamma = 0.05 # 혹시 최소 수익률
# overlap_ratio = 0.4

for _ in range(len_df2):
  for index, row in df2.iterrows():
    sort_last = []
    print(f'fellow : {index}')
    i_org = df2.loc[index,'원래시간(분)']
    i_rou = df2.loc[index,'경유시간(분)']
    a_org = df2.loc[index,'타깃원래시간(분)']
    a_rou = df2.loc[index,'타깃경유시간(분)']
    sort_last.append(max((i_rou-i_org)/i_org,(a_rou-a_org)/a_org))
    ratio_index[max((i_rou-i_org)/i_org,(a_rou-a_org)/a_org)] = index

    fellow_user = df2.loc[ratio_index[min(sort_last)]]['ID']
    fellow_index = ratio_index[min(sort_last)]

    a_ind_pred = int(4800 + (df2.loc[fellow_index,'타깃원래거리(km)']-1.6)*100/1.31 + (df2.loc[fellow_index,'타깃원래시간(분)']-96/23)*200)
    i_ind_pred = int(4800 + (df2.loc[fellow_index,'원래거리(km)']-1.6)*100/1.31 + (df2.loc[fellow_index,'원래시간(분)']-96/23)*200)
    driver_rou_pred = int(4800 + (df2.loc[fellow_index,'총경유거리(km)']-1.6)*100/1.31 + (df2.loc[fellow_index,'총이동시간(분)']-96/23)*200)

    det = a_ind_pred*(1-alpha_min-beta_min*(a_rou-a_org)/a_org)+ i_ind_pred*(1-alpha_min-beta_min*(i_rou-i_org)/i_org) - driver_rou_pred*(1+gamma)

    # if (a_ind_pred + i_ind_pred - driver_rou_pred)/driver_rou_pred > overlap_ratio:
    if (det > 0):
      print(f"최종적으로 'A'와 '{fellow_user}'을(를) 매칭해준다. ( {det:.2f} )")
      print()
      break
    else:
      df2 = df2.drop(index, axis=0)
      print(f"{fellow_user}, : {det:.2f} => 최종 매칭 X")
      print()
  # if (a_ind_pred + if_ind_pred - driver_rou_pred)/driver_rou_pred > overlap_ratio:
  if (det > 0):
    break

fellow : 24
X, : -1723.45 => 최종 매칭 X

fellow : 41
OO, : -4184.37 => 최종 매칭 X

fellow : 49
WW, : -1394.42 => 최종 매칭 X

fellow : 35
II, : -3446.68 => 최종 매칭 X

fellow : 34
최종적으로 'A'와 'HH'을(를) 매칭해준다. ( 3792.83 )



### Step6) 최종 select된 타깃동승객과 매칭동승객 df1에서 제거 후 df3에 추가

In [226]:
fellow_index = ratio_index[min(sort_last)]
fellow_info = df2.loc[fellow_index]
# df1에 매칭된 승객 제거
df1 = df1.drop(index=[target_index,fellow_index])

print(fellow_index)
print(fellow_info)

34
ID                                                           HH
출발희망시각                                      2023-03-10 17:20:00
최대대기시각                                      2023-03-10 18:00:00
시간범위                                               (1040, 1080)
출발위치                                                 (300, 300)
도착위치                                       (298.6848, 738.5939)
타깃경유시간(분)                                             86.833529
타깃경유거리(km)                                            33.286186
타깃원래시간(분)                                             49.613614
타깃원래거리(km)                                            19.018552
경유시간(분)                                               61.638325
경유거리(km)                                              23.628024
원래시간(분)                                               61.638325
원래거리(km)                                              23.628024
총이동시간(분)                                              86.833529
총경유거리(km)                            

In [227]:
print(target_info)

ID                          A
출발희망시각    2023-03-10 17:30:00
최대대기시각    2023-03-10 18:10:00
시간범위             (1050, 1090)
출발위치               (300, 100)
도착위치      (97.3956, 572.4011)
Name: 1, dtype: object


In [126]:
target_info.출발위치

(300, 100)

In [229]:
# df3에 추가
target_id = target_info.ID
fellow_id = fellow_info.ID
routing_point = [fellow_info.이동경로]
routing_order = [fellow_info.승하차순서]


df3 = pd.DataFrame(dict(zip(['타킷동승자(target)','매칭동승자(fellow)','이동경로','승하차순서'],[target_id, fellow_id, routing_point, routing_order])))

df3

,타킷동승자(target),매칭동승자(fellow),이동경로,승하차순서
0,A,HH,"[(300, 100), (300, 100), (298.6848, 738.5939),...","[i_start, i_start, a_end, i_end]"


# SETTLEMENT ALGORITHM

In [238]:
a_ind_pred = int(4800 + (df2.loc[fellow_index,'타깃원래거리(km)']-1.6)*100/1.31 + (df2.loc[fellow_index,'타깃원래시간(분)']-96/23)*200)
i_ind_pred = int(4800 + (df2.loc[fellow_index,'원래거리(km)']-1.6)*100/1.31 + (df2.loc[fellow_index,'원래시간(분)']-96/23)*200)
driver_rou_pred = int(4800 + (df2.loc[fellow_index,'총경유거리(km)']-1.6)*100/1.31 + (df2.loc[fellow_index,'총이동시간(분)']-96/23)*200)

print(a_ind_pred, i_ind_pred, driver_rou_pred)

15217 17974 23750


In [239]:
C1 = a_ind_pred
C2 = (a_rou-a_org)/a_org
C3 = i_ind_pred
C4 = (i_rou-i_org)/i_org
C5 = driver_rou_pred
eplison = 0.03 # 실제 요금/예상요금 비율
gamma = 0.05 # 매칭당 혹시의 이익률

LB = 0.2
UB = 0.5

try:
  # Create a new model
  m = gp.Model('cost_optimization1')

  # Create variables
  alpha = m.addVar(vtype=GRB.CONTINUOUS, name='alpha')
  beta = m.addVar(vtype=GRB.CONTINUOUS, name='beta')

  # Set objective
  m.setObjective(C1*(1-(alpha+beta*C2))+C3*(1-(alpha+beta*C4))-(1+gamma)*C5, GRB.MINIMIZE)

  # Add constraint
  m.addConstr(max(C2,C4)*beta-UB*alpha <= 0,'const_1')
  m.addConstr(max(C2,C4)*beta-LB*alpha >= 0,'const_2')
  m.addConstr((C1*(1-(alpha+beta*C2))+C3*(1-(alpha+beta*C4))-(1+gamma)*C5) >= 0,'const_3')

  m.addConstr(alpha <= 0.5)
  m.addConstr(alpha >= 0)
  m.addConstr(beta <= 0.5)
  m.addConstr(beta >= 0)
  #m.addConstr(gamma <= 1)

  # Optimize model
  m.optimize()

  for v in m.getVars():
    print('%s : %g' % (v.varName, v.x))
    globals()[v.varName] = v.x
  print('Obj : %g' % m.objVal)

except gp.GurobiError as e:
  print('Error code ' + str(e.errno) + ': ' + str(e))

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 2 columns and 10 nonzeros
Model fingerprint: 0x273fb45e
Coefficient statistics:
  Matrix range     [2e-01, 3e+04]
  Objective range  [1e+04, 3e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 8e+03]
Presolve removed 4 rows and 0 columns
Presolve time: 0.01s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.9429264e+03   7.589556e+00   0.000000e+00      0s
       2    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  0.000000000e+00
alpha : 0.227781
beta : 0.0607257
Obj : 0


In [240]:
print(f'타깃동승자   {alpha*100:.2f}+{beta*C2*100:.2f} => {(alpha+beta*C2)*100:.2f}% 할인')
print(f'동승자       {alpha*100:.2f}+{beta*C4*100:.2f}  => {(alpha+beta*C4)*100:.2f}% 할인')
print(f'타깃깃동승자 {C1}원 >> {(1-alpha+beta*C2)*C1:.0f}원')
print(f'동승자       {C3}원 >> {(1-alpha+beta*C4)*C3:.0f}원')
print(f'혹시가 얻는 최종 이익 >> {(1-alpha+beta*C2)*C1+(1-alpha+beta*C4)*C3-(1+eplison)*C5:.0f} 원')
print()
print(f'최적화 후 objective value: {(1-alpha-beta*C2)*C1+(1-alpha-beta*C4)*C3-(1+gamma)*C5:.0f} 원')

타깃동승자   22.78+4.56 => 27.33% 할인
동승자       22.78+0.00  => 22.78% 할인
타깃깃동승자 15217원 >> 12444원
동승자       17974원 >> 13880원
혹시가 얻는 최종 이익 >> 1861 원

최적화 후 objective value: 0 원
